#### Objective : To try hand at feature engineering 
- learn the importance 
- some techniques


Dataset : https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data 

In [2]:
import pandas as pd
df_train = pd.read_csv(r'./Data/feature_engineering/train.csv',index_col='id')

In [3]:
print(df_train.info())
print(df_train.describe())

<class 'pandas.core.frame.DataFrame'>
Index: 159571 entries, 0000997932d777bf to fff46fc426af1f9a
Data columns (total 7 columns):
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(1)
memory usage: 9.7+ MB
None
               toxic   severe_toxic        obscene         threat  \
count  159571.000000  159571.000000  159571.000000  159571.000000   
mean        0.095844       0.009996       0.052948       0.002996   
std         0.294379       0.099477       0.223931       0.054650   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       0.000000       0.000000   
50%         0.000000       0.000000       0.000000       0.000000   
75%         0.000000       0.000000       0.000000       0

In [4]:
# checking the null values
print(df_train.isnull().sum())

comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64


In [5]:
df_train.tail()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [6]:
df_train['comment_text'].tail()

id
ffe987279560d7ff    ":::::And for the second time of asking, when ...
ffea4adeee384e90    You should be ashamed of yourself \n\nThat is ...
ffee36eab5c267c9    Spitzer \n\nUmm, theres no actual article for ...
fff125370e4aaaf3    And it looks like it was actually you who put ...
fff46fc426af1f9a    "\nAnd ... I really don't think you understand...
Name: comment_text, dtype: object

In [7]:
df_train.loc[:,['toxic','severe_toxic','obscene','threat','insult','identity_hate']].apply(sum, axis=0)

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

In [8]:
#CLEANING UP THE TEXT
#Function to clean up the text
def standardizetext(df, textfield):
    df[textfield] = df[textfield].str.replace(r"http\S+", "")
    df[textfield] = df[textfield].str.replace(r"http", "")
    df[textfield] = df[textfield].str.replace(r"@\S+", "")
    df[textfield] = df[textfield].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[textfield] = df[textfield].str.replace(r"@", "at")
    df[textfield] = df[textfield].str.lower()
    return df

In [9]:
df_train = standardizetext(df_train, "comment_text")
df_train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
0000997932d777bf,explanation\nwhy the edits made under my usern...,0,0,0,0,0,0
000103f0d9cfb60f,d'aww! he matches this background colour i'm s...,0,0,0,0,0,0
000113f07ec002fd,"hey man, i'm really not trying to edit war it...",0,0,0,0,0,0
0001b41b1c6bb37e,"""\nmore\ni can't make any real suggestions on ...",0,0,0,0,0,0
0001d958c54c6e35,"you, sir, are my hero any chance you remember...",0,0,0,0,0,0


In [11]:
X = df_train['comment_text']
Y = df_train.drop(['comment_text'], axis=1)

In [12]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.8, test_size=0.2, random_state=23)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
tf_vectorizer = TfidfVectorizer(decode_error='ignore',  min_df=0.009,stop_words='english')

In [23]:
X_train = tf_vectorizer.fit_transform(X_train)
X_test = tf_vectorizer.transform(X_test)

In [27]:
from sklearn.multiclass import OneVsRestClassifier

In [28]:
from sklearn.svm import LinearSVC

In [29]:
model = OneVsRestClassifier(LinearSVC())
model.fit(X_train, Y_train)

OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1)

In [30]:
# Accuracy score on test dataset
model.score(X_test,Y_test)

0.9073476421745261

In [35]:
# Accuracy score on train dataset
model.score(X_train,Y_train)

0.9078930876731215

In [33]:
model.predict(X_test[0:10])

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0]])

In [34]:
Y_test[0:10]

,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,
88d640ab9223398d,0,0,0,0,0,0
9f9b412e0a99e6dc,0,0,0,0,0,0
323848f9f866c47e,0,0,0,0,0,0
90bd2ed36352d163,0,0,0,0,0,0
fe77e59403ac395e,0,0,0,0,0,0
17e22e51f52ae4a4,0,0,0,0,0,0
b1d80e1ab696b108,0,0,0,0,0,0
9329101468aa1b93,0,0,0,0,0,0
4c4cae190527b2a1,0,0,0,0,0,0


In [36]:
from sklearn.multiclass import OneVsOneClassifier

In [37]:
model_one_v_one = OneVsOneClassifier(LinearSVC())

In [45]:
# model_one_v_one.fit(X_train,Y_train)
# model_one_v_one.score(X_test, Y_test)

###  Using Multiple Output regressor -

Multiple out regressor fits multiple regressor - this stratergy fits one regressor per target. Since each traget is represented by exactly one regressor  it is possible to drive strength out of it.

In [40]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [42]:
multi_model = MultiOutputClassifier(GradientBoostingClassifier())

In [44]:
%%time
multi_model.fit(X_train, Y_train)


Wall time: 7min 29s


MultiOutputClassifier(estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
           n_jobs=1)

In [46]:
multi_model.score(X_test, Y_test)

0.905342315525615

In [47]:
multi_model.score(X_train, Y_train)

0.9102196528169455

###  UnderStanding TfidfVectorizer

In [27]:
x_df_train = df_train['comment_text']

x_df_train.isnull().any()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [56]:

tf_vectorizer = TfidfVectorizer(decode_error='ignore',  min_df=0.009,stop_words='english')

In [57]:
trans = tf_vectorizer.fit_transform(x_df_train)

In [58]:
print(trans.shape)
print(tf_vectorizer.get_feature_names())
print(len(tf_vectorizer.get_feature_names()))

(159571, 454)
['10', '100', '11', '12', '14', '15', '20', '2005', '2006', '2007', '2008', '24', 'able', 'according', 'account', 'actual', 'actually', 'add', 'added', 'adding', 'address', 'admin', 'administrator', 'admins', 'ago', 'agree', 'allowed', 'american', 'answer', 'anti', 'appear', 'appears', 'appreciate', 'appropriate', 'aren', 'argument', 'article', 'articles', 'ask', 'asked', 'attack', 'attacks', 'attention', 'automatically', 'aware', 'away', 'bad', 'based', 'believe', 'best', 'better', 'big', 'bit', 'block', 'blocked', 'book', 'books', 'called', 'came', 'care', 'case', 'category', 'certain', 'certainly', 'change', 'changed', 'changes', 'check', 'cheers', 'cited', 'claim', 'claims', 'clear', 'clearly', 'come', 'comment', 'comments', 'common', 'community', 'completely', 'consensus', 'consider', 'considered', 'contact', 'content', 'continue', 'contribs', 'contributing', 'contributions', 'copyright', 'correct', 'country', 'course', 'create', 'created', 'criteria', 'current', 'cu

In [61]:
print(tf_vectorizer.get_params())

{'analyzer': 'word', 'binary': False, 'decode_error': 'ignore', 'dtype': <class 'numpy.int64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'max_df': 1.0, 'max_features': None, 'min_df': 0.009, 'ngram_range': (1, 1), 'norm': 'l2', 'preprocessor': None, 'smooth_idf': True, 'stop_words': 'english', 'strip_accents': None, 'sublinear_tf': False, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None, 'use_idf': True, 'vocabulary': None}


In [59]:
print(tf_vectorizer.get_stop_words())

frozenset({'get', 'part', 'that', 'three', 'yourself', 'seemed', 'bottom', 'cant', 'may', 'my', 'formerly', 'hasnt', 'throughout', 'wherever', 'amongst', 'another', 'couldnt', 'might', 'such', 'been', 'he', 'otherwise', 'will', 'anywhere', 'yet', 'via', 'cannot', 'mill', 'besides', 'towards', 'amount', 'back', 'nine', 'side', 'empty', 'everything', 'either', 'sixty', 'ten', 'whenever', 'you', 'eg', 'none', 'something', 'therein', 'front', 'ever', 'for', 'these', 'else', 'too', 'where', 'who', 'much', 'moreover', 'noone', 'former', 'full', 'over', 'until', 'thereupon', 'beforehand', 'yours', 'or', 'own', 'name', 'the', 'hereby', 'latterly', 'co', 'more', 'should', 'whoever', 'in', 'enough', 'which', 'very', 'five', 'beyond', 'thence', 'meanwhile', 'move', 'except', 'are', 'being', 'no', 'after', 'again', 'themselves', 'all', 'per', 'often', 'then', 'even', 'anyway', 'someone', 'two', 'by', 'somewhere', 'ourselves', 'hundred', 'anything', 'from', 'am', 'both', 'how', 'namely', 'onto', 'm

### Summary -- Take Aways


#### References :
https://www.kaggle.com/eikedehling/feature-engineering

[Multi Class Classification](# https://www.youtube.com/watch?v=y0wQK3mnrNY)